In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import json

* WADI_normaldata.csv: delete the first 4 lines of WADI_14days.csv  
* sensor_map.txt: map the tedious sensor names to "sensor0": "sensor126", sensor_map.txt stores the map

* use sensor0 data to do univariate autoencoder
But the result is very bad. 

In [10]:
def load_normal_data():
    data_df = pd.read_csv("../data/WADI/WADI_normaldata.csv")
    sensor_map = {}
    with open("../data/WADI/sensor_map.txt", "r") as f:
        sensor_map = json.loads(f.read().replace("'", '"'))
    data_df = data_df.rename(columns = sensor_map)
    return data_df

In [11]:
def load_anomaly_data():
    data_df = pd.read_csv("../data/WADI/WADI_attackdata_labelled.csv")
    sensor_map = {}
    with open("../data/WADI/sensor_map.txt", "r") as f:
        sensor_map = json.loads(f.read().replace("'", '"'))
    
    data_df = data_df.rename(columns = sensor_map)
    return data_df

In [12]:
normal_data = load_normal_data()
anomaly_data = load_anomaly_data()
    

In [14]:
# try sensor 0 first

In [22]:
sensor0_normal = np.array(normal_data["sensor0"])
sensor0_anomaly = np.array(anomaly_data["sensor0"])

In [26]:
sensor0_max = max(max(sensor0_normal), max(sensor0_anomaly)).astype("double")
sensor0_min = min(min(sensor0_normal), min(sensor0_anomaly)).astype("double")
sensor0_normal_normalized = (sensor0_normal-sensor0_min)/(sensor0_max-sensor0_min)
sensor0_anomaly_normalized = (sensor0_anomaly-sensor0_min)/(sensor0_max-sensor0_min)

In [27]:
normal_len = len(sensor0_normal)

In [20]:
#generate train_x

In [112]:
sample_size = 20
train_size = 10000
sample_step = np.floor(normal_len/train_size).astype(int)

In [113]:
train_x = np.zeros((train_size, 1, sample_size), dtype="double")

In [114]:
for i in range(train_size):
    train_x[i][0] = sensor0_normal_normalized[i*sample_step: (i*sample_step+sample_size)]

In [115]:
train_y = train_x[:,0,:].copy()

In [116]:
anomaly_begin = ((1*24+11)*60+7)*60+46
anomaly_end = ((1*24+11)*60+12)*60+15
test_anomaly_size = 20
sample_step_test_anomaly = np.floor((anomaly_end-anomaly_begin-sample_size)/test_anomaly_size).astype(int)

In [117]:
test_x_anomaly = np.zeros((test_anomaly_size, 1, sample_size), dtype="double")
for i in range(test_anomaly_size):
    test_x_anomaly[i][0]=sensor0_anomaly_normalized[\
                        i*sample_step_test_anomaly:(i*sample_step_test_anomaly+sample_size)]
test_y_anomaly = test_x_anomaly[:,0,:].copy()

In [118]:
test_x_normal = []
test_x_index = np.random.choice(len(sensor0_anomaly_normalized)-sample_size, 100)
for i in test_x_index:
    if((i+sample_size)>=anomaly_begin and (i<=anomaly_end)):
        continue
    else:
        test_x_normal.append([sensor0_anomaly_normalized[i:i+sample_size]])
test_x_normal = np.array(test_x_normal)
test_y_normal = test_x_normal[:,0,:].copy()

In [123]:
model3 = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(1,20)),
    tf.keras.layers.Dense(14, activation=tf.nn.relu),
    tf.keras.layers.Dense(9, activation=tf.nn.relu),
    tf.keras.layers.Dense(9, activation=tf.nn.relu),
    
    tf.keras.layers.Dense(9, activation=tf.nn.relu),
    tf.keras.layers.Dense(14, activation=tf.nn.relu),
    tf.keras.layers.Dense(20, activation=tf.nn.softmax)
])
model3.compile(optimizer="Adam",
              loss=tf.keras.losses.mean_squared_error,
               metrics=["mse"])
model3.fit(train_x, train_y, epochs=10)


Epoch 1/10
10000/10000 [==============================] - 1s 111us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 2/10
10000/10000 [==============================] - 1s 73us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 3/10
10000/10000 [==============================] - 1s 73us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 4/10
10000/10000 [==============================] - 1s 77us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 5/10
10000/10000 [==============================] - 1s 70us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 6/10
10000/10000 [==============================] - 1s 66us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 7/10
10000/10000 [==============================] - 1s 73us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 8/10
10000/10000 [==============================] - 1s 71us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 9/10
10000/10000 [==============================] - 1s 70

In [124]:
model3.fit(train_x, train_y, epochs=10)

Epoch 1/10
10000/10000 [==============================] - 1s 71us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 2/10
10000/10000 [==============================] - 1s 72us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 3/10
10000/10000 [==============================] - 1s 74us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 4/10
10000/10000 [==============================] - 1s 73us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 5/10
10000/10000 [==============================] - 1s 68us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 6/10
10000/10000 [==============================] - 1s 68us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 7/10
10000/10000 [==============================] - 1s 70us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 8/10
10000/10000 [==============================] - 1s 75us/sample - loss: 0.0463 - mean_squared_error: 0.0463
Epoch 9/10
10000/10000 [==============================] - 1s 72u

In [125]:
print("normal: ", model3.evaluate(test_x_normal, test_y_normal))

100/100 [==============================] - 0s 2ms/sample - loss: 0.0514 - mean_squared_error: 0.0514
normal:  [0.05136388391256332, 0.05136389]


In [126]:
print("anomaly: ", model3.evaluate(test_x_anomaly, test_y_anomaly))

20/20 [==============================] - 0s 75us/sample - loss: 0.0436 - mean_squared_error: 0.0436
anomaly:  [0.043597590178251266, 0.04359759]
